<a href="https://colab.research.google.com/github/MevrouwHelderder/final_assignment/blob/main/2nd_Try_Final_Assignment_Energy_production_and_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final assignment - Energy production and consumption



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext google.colab.data_table

In [3]:
path_base = "https://raw.githubusercontent.com/MevrouwHelderder/final_assignment/main/"

paths = {
    "energy_total" : path_base + "owid-energy-data.csv",
    "co2_greenhouse" : path_base + "owid-co2-data.csv",
    "net_zero_pledge" : path_base + "net-zero-targets.csv",
    # "agri_land_use_total" : path_base + "total-agricultural-area-over-the-long-term.csv",
    # "grazing" : path_base + "grazing-land-use-over-the-long-term.csv",
    # "cropland" : path_base + "cropland-use-over-the-long-term.csv"
}

dataframes = {}

for key, value in paths.items():
  dataframes[key] = pd.read_csv(value)

# energy_total_df = dataframes["energy_total"]
co2_greenhouse_df = dataframes["co2_greenhouse"]
net_zero_pledge_df = dataframes["net_zero_pledge"]
# agri_land_use_total_df = dataframes["agri_land_use_total"]
# grazing_df = dataframes["grazing"]
# cropland_df = dataframes["cropland"]

# Exploration
Let's first explore the datasets to see what we are working with.
It might also be better to weed out data that is not needed now and perhaps combine data that is needed.

In [ ]:
# General: 
# country	
# year
# iso_code	
# population	
# gdp

# co2	
# co2_growth_abs	
# co2_growth_prct	

# co2_including_luc	Annual 
# co2_including_luc_growth_abs	
# co2_including_luc_growth_prct	
# co2_including_luc_per_capita	
# co2_including_luc_per_gdp	
# co2_including_luc_per_unit_energy


# co2_per_capita	
# co2_per_gdp	
# co2_per_unit_energy	

# consumption_co2	
# consumption_co2_per_capita
# consumption_co2_per_gdp	


# energy_per_capita	
# energy_per_gdp	
# primary_energy_consumption	

# share_global_co2	
# share_global_co2_including_luc	
# share_global_cumulative_co2	
# share_global_cumulative_co2_including_luc	

# trade_co2	
# trade_co2_share	

# First let's see what columns can stay

In [39]:
# consumption_co2 has been adjusted for trade
# co2 has not been adjusted for

co2_df = co2_greenhouse_df.loc[:,['country',	
'year',
'iso_code',	
'co2',	
'co2_including_luc',	
'co2_including_luc_per_capita',	
'co2_per_capita',	
'consumption_co2',	
'consumption_co2_per_capita', 
'energy_per_capita',	
'primary_energy_consumption',	
'trade_co2',	
'population',	
'gdp',
'co2_including_luc_per_gdp',	
'co2_per_gdp',	
'consumption_co2_per_gdp',	
'energy_per_gdp']]

# show only the true countries, not the combinations like continents
co2_df = co2_df[~co2_df['iso_code'].isnull()].copy()

# Add a column with the gdp per capita to totals
co2_df['gdp_per_capita'] = (co2_df['gdp']/co2_df['population']).round(3)

# Changing 0 to NaN to prevent infinity
co2_df = co2_df.replace (0, np.nan)

# Setting the index
co2_df = co2_df.set_index("year")
co2_df.index = pd.to_datetime(co2_df.index, format="%Y", errors="coerce")

co2_df = co2_df.drop(['iso_code', 'gdp', 'population'], axis=1)

# Taking the last 20 years 
co2_df_twenty_years = co2_df[(co2_df.index.year >= 2000)].copy(deep=True)
co2_df_all_years = co2_df.copy(deep=True)
co2_one_year = co2_df[(co2_df.index.year == 2015)].dropna().copy(deep=True)

In [40]:
co2_one_year.head(5)

,country,co2,co2_including_luc,co2_including_luc_per_capita,co2_per_capita,consumption_co2,consumption_co2_per_capita,energy_per_capita,primary_energy_consumption,trade_co2,co2_including_luc_per_gdp,co2_per_gdp,consumption_co2_per_gdp,energy_per_gdp,gdp_per_capita
year,,,,,,,,,,,,,,,
2015-01-01,Albania,4.712,5.885,2.042,1.635,5.917,2.053,11824.314,34.083,1.204,0.193,0.155,0.194,1.120,10561.916
2015-01-01,Argentina,191.744,282.318,6.527,4.433,200.071,4.625,23037.494,996.534,8.327,0.333,0.226,0.236,1.175,19600.382
2015-01-01,Armenia,5.469,6.532,2.269,1.900,5.885,2.044,14125.896,40.663,0.416,0.222,0.186,0.200,1.385,10201.253
2015-01-01,Australia,401.793,437.700,18.375,16.868,382.345,16.051,67151.320,1599.561,-19.448,0.382,0.351,0.334,1.397,48084.605
2015-01-01,Austria,66.349,64.480,7.461,7.677,89.202,10.321,45737.727,395.285,22.853,0.180,0.185,0.248,1.101,41537.907
